### 从UCI加载数据，转化格式，转换为数字形式，并保存为mat数据 
X为特征数据：num_instances*features
Y为lable：(num_instances,1) 是一个列向量

## numpy转mat的demo

### 测试保存的mat

In [ ]:
from utils.dataset_utils import get_classes_indexes_counts
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import confusion_matrix
from utils.dataset_utils import get_classes_indexes_counts
# 导入必要的库
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
import scipy.io as sio  # 从.mat文件中读取数据集
import numpy as np
# 加载鸢尾花数据集
# data = load_iris()
mat_data = sio.loadmat('Satellite.mat')
X = mat_data['X']  # 特征
y = mat_data['Y'][:, 0]  # 标签
classes, counts = get_classes_indexes_counts(y)
print(counts)
print(X.shape)
print(y.shape)
# 将数据集分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# classes, counts = get_classes_indexes_counts(np.argmax(y_test, axis=1))
classes, counts = get_classes_indexes_counts(y_test)
print(counts)
# 数据标准化
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
# One-hot encode target variable 强制将类别转换为0-1序列，0表示不是该类，1表示属于该类

# 构建并训练MLP模型
mlp = MLPClassifier(hidden_layer_sizes=(15,), max_iter=200, random_state=42,learning_rate_init=0.001)
# mlp.fit(X_train, np.argmax(y_train, axis=1))
mlp.fit(X_train, y_train)
print(mlp.classes_)
index_pred_proba = mlp.predict_proba(X_test)
# 预测和评估模型
y_pred = mlp.predict(X_test)
#y_test_labels = np.argmax(y_test, axis=1)
# y_pred_labels = np.argmax(y_pred, axis=1)
y_pred_labels = y_pred
# 输出结果
print("准确率:", accuracy_score(y_test, y_pred_labels))
print("\n分类报告:\n", classification_report(y_test, y_pred_labels))
# 打印混淆矩阵
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_labels))

print(index_pred_proba.shape)
res=np.sum(index_pred_proba, axis=1)
print(res)

### k-folds交叉验证


In [1]:
from scipy.stats import gmean
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
# 导入必要的库
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from utils.dataset_utils import get_classes_indexes_counts
import scipy.io as sio  # 从.mat文件中读取数据集
# 加载鸢尾花数据集
# iris = load_iris()
# 数据集

mat_data = sio.loadmat('../data/dataset/Connect4.mat')
# 提取变量
dataset_x = mat_data['X']
dataset_y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
# X, y = iris.data, iris.target
X, y = mat_data['X'], mat_data['Y'][:, 0]
# 统计每个类别的个数
classes, counts = get_classes_indexes_counts(y)  #np.argmax(y_onehot, axis=1)找最大值的索引，将0-1序列转化为0,1,2,3......的整数标签
print("每种类别的分布：", counts)
print("#########################5-folds交叉验证#########################")
# 定义MLP模型
mlp_model = MLPClassifier(hidden_layer_sizes=(20,), max_iter=1000, random_state=42,learning_rate_init=0.001)

# 定义5折交叉验证
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 使用交叉验证进行预测
y_pred = cross_val_predict(mlp_model, X, y, cv=kf)
y_pred_proba = cross_val_score(mlp_model, X, y, cv=kf)
# 输出分类结果报告
report = classification_report(y, y_pred)
#print(report)
# 计算 ROC AUC（ovo+macro）
#auc_ovo_macro = roc_auc_score(y, y_pred_proba, multi_class="ovo", average="macro")
cm = confusion_matrix(y, y_pred)
# 计算每类召回率（每类正确预测个数 / 该类总数）
recall_per_class = cm.diagonal() / cm.sum(axis=1)

# 计算G-Mean
geometric_mean = gmean(recall_per_class)
#print(f"最终的分类结果：Recall_Per_Class{recall_per_class}，Gmean：{geometric_mean}，mAUC：{auc_ovo_macro}")
print(f"最终的分类结果：Recall_Per_Class{recall_per_class}，Gmean：{geometric_mean}")


x_train, x_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.3, random_state=5)
# 统计每个类别的个数 
classes_train, counts_train = get_classes_indexes_counts(y_train)
print("训练集每种类别的分布：", counts_train)
classes_test, counts_test = get_classes_indexes_counts(y_test)
print("测试集每种类别的分布：", counts_test)
print("#########################NoS原始数据集训练预测结果#########################")
mlp = MLPClassifier(hidden_layer_sizes=(20,), max_iter=1000, random_state=42,learning_rate_init=0.001)
mlp.fit(x_train, y_train)

# 使用训练数据进行预测
ind_pred = mlp.predict(x_test)  # 计算accuracy、PPV
index_pred_proba = mlp.predict_proba(x_test)  # 计算mAUC
######################G-mean#########################
# 计算混淆矩阵
cm = confusion_matrix(y_test, ind_pred)

# 计算每类召回率（每类正确预测个数 / 该类总数）
recall_per_class = cm.diagonal() / cm.sum(axis=1)

# 计算G-Mean
geometric_mean = gmean(recall_per_class)
######################mAUC#######################
# 计算 ROC AUC（ovo+macro）
auc_ovo_macro = roc_auc_score(y_test, index_pred_proba, multi_class="ovo", average="macro")
print(f"最终的分类结果：Recall_Per_Class{recall_per_class}，Gmean：{geometric_mean}，mAUC：{auc_ovo_macro}")

每种类别的分布： [ 6449 16635 44473]
#########################5-folds交叉验证#########################
最终的分类结果：Recall_Per_Class[0.00527214 0.58400962 0.91664605]，Gmean：0.14131971008904698


NameError: name 'train_test_split' is not defined